# Browse Workflows

In [ ]:
from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)
    
from aiida.orm.querybuilder import QueryBuilder
from aiida.orm.calculation.work import WorkCalculation

import ipywidgets as ipw
import datetime
from pprint import pprint
from collections import OrderedDict
from IPython.display import display, clear_output

In [ ]:
age_range = ipw.IntRangeSlider(
                value=[0, 7],
                min=0,
                max=100,
                step=1,
                description='age in days:',
                layout=ipw.Layout(width="900px"))

def on_age_change(c):
    update_search()

age_range.observe(on_age_change, names='value')
display(age_range)

In [ ]:
def update_search():
    min_age = datetime.datetime.now() - datetime.timedelta(days=age_range.value[0])
    max_age = datetime.datetime.now() - datetime.timedelta(days=age_range.value[1])

    qb = QueryBuilder()
    qb.append(WorkCalculation, filters={
                                        'ctime':{'and':[{'<=': min_age},{'>': max_age}]},
                                        'attributes': {'!has_key': 'source_code'}
              })
    qb.order_by({WorkCalculation:{'ctime':'desc'}})

    options = OrderedDict()
    for n in qb.iterall():
        n = n[0]
        ctime = n.ctime.strftime("%Y-%m-%d %H:%M")
        nsteps = len(n.get_outputs())
        label = "PK %d; %s; %s; steps: %d" %(n.pk, ctime, n.get_attr("_process_label"), nsteps)
        if "structure" in n.get_inputs_dict():
            s = n.inp.structure
            label += "   structure: PK: %s %s"%(s.pk, s.description)
        options[label] = n
    wf_select.options = options

In [ ]:
def on_wf_changed(c):
    update_wf()

wf_out = ipw.Output(layout=ipw.Layout(border="2px solid black"))
wf_select = ipw.Select(layout=ipw.Layout(height="300px", width="900px"))
wf_select.observe(on_wf_changed, names='value')
display(wf_select, wf_out)

In [ ]:
def update_wf():
    with wf_out:
        clear_output()
        node = wf_select.value
        if not node:
            print("no selection")
            return

        print("NODE:")
        print(repr(node))
        
        print("\n\nINPUTS:")
        pprint(node.get_inputs_dict())
        
        print("\n\nREPORT:")
        ! verdi work report $node.pk
        
        print("\n\nTREE:")
        ! verdi work tree $node.pk
        
        print("\n\nSTATE:")
        ! verdi work tree --node-label state $node.pk

In [ ]:
update_search()